In [10]:
import os
import fluidsynth
import time

# Set up FluidSynth path (if needed)
fluidsynth_bin_path = r"C:\tools\fluidsynth\bin"
if os.path.exists(fluidsynth_bin_path):
    os.add_dll_directory(fluidsynth_bin_path)

# Initialize FluidSynth
fs = fluidsynth.Synth()
fs.start(driver="dsound")  # Use DirectSound for Windows

# Load a SoundFont (.sf2 file)
soundfont_path = r"C:\tools\fluidsynth\MuseScore_General.sf2"  # Replace with the actual file path
if not os.path.exists(soundfont_path):
    raise FileNotFoundError(f"SoundFont not found: {soundfont_path}")

sfid = fs.sfload(soundfont_path)
fs.program_select(0, sfid, 0, 0)  # Select Piano (Bank 0, Preset 0)

# Define "Für Elise" melody (right hand) & chords (left hand)
fur_elise_melody = [
    76, 75, 76, 75, 76, 71, 74, 72, 69, 69, 69, 72, 76, 79, 81, 76, 79, 74, 77, 72, 75, 69, 72
]
fur_elise_chords = [
    (52, 56, 59), (52, 57, 60), (52, 56, 59), (52, 57, 60),  # Alternating chords
    (52, 56, 59), (52, 57, 60), (52, 56, 59), (52, 57, 60),
    (48, 52, 55), (47, 50, 54), (48, 52, 55), (47, 50, 54),
    (48, 52, 55), (47, 50, 54), (48, 52, 55), (47, 50, 54)
]

tempo = 100  # BPM (Für Elise is around 100 BPM)

# Play Für Elise melody and chords together
for i, note in enumerate(fur_elise_melody):
    # Play left-hand chord (loop back if needed)
    chord = fur_elise_chords[i % len(fur_elise_chords)]
    for chord_note in chord:
        fs.noteon(0, chord_note, 80)  # Play chord

    # Play right-hand melody
    fs.noteon(0, note, 100)
    time.sleep(60 / tempo)  # Convert BPM to time delay
    fs.noteoff(0, note)

    # Stop left-hand chord notes
    for chord_note in chord:
        fs.noteoff(0, chord_note)

# Stop FluidSynth
fs.delete()


In [16]:
#x=[1,2,4,6,8,10,12,14,16,18,20]
#y=[0.6,0.9,1.5,1.9,2.1,2.6,2.9,3.7,3.5,3.7,3.8]
x = [1.6,9.4,15.5,20,22,35.5,43,40.5,33]
y = [240,181,193,155,172,110,113,75,94]

n = len(x)

def avg(a):
    sum=0
    for num in a:
        sum+=num
    avg = sum/n
    return avg

avg_x = avg(x)
avg_y = avg(y)

b_1_upper = 0
for i in range(len(x)):
    b_1_upper += (x[i]-avg_x)*(y[i]-avg_y)

b_1_lower = 0
for num_x in x:
    b_1_lower += (num_x-avg_x)**2

b_1 = b_1_upper/b_1_lower

print("b1: ",b_1_upper,b_1_lower,b_1)

b_0 = avg_y-(b_1*avg_x)
print("b0:",b_0)

def model(x):
    y_pred = b_0+b_1*(x)
    return y_pred
print(model(15))

avg_y_pred = 0.0
for num in x:
    y_pred = model(num+3)
    avg_y_pred += y_pred
avg_y_pred = avg_y_pred/n
print("avg_y_pred",avg_y_pred)

print(model(6))

ssr = 0
for i in x:
    ssr += (model(i)-avg_y)**2

sst = 0
for i in y:    
    sst += (i-avg_y)**2

R_square = ssr/sst
print(ssr,sst,R_square)

b1:  -5794.099999999999 1651.42 -3.508556272783422
b0: 234.07073979430496
181.44239570255363
avg_y_pred 137.58544229276083
213.01940215760442
20328.925900134418 23116.88888888889 0.8793971367793136
